# Prototype_2
### 추출해와서 출력하기

#### Requirements

In [6]:
import os
import glob

import numpy as np
import pandas as pd
import json

import mediapipe as mp
import cv2
import pytube

#### Prototype

In [22]:
class JustDDance:
    def __init__(self):
        self.__download_path = "./video/"
        self.__keyporint_extraction_path = "./keypoint_extraction/"
        self.__dance_fps = None
        self.__dance_shape = None
        self.__user_fps = None
        self.__user_shape = None
    
    def download_video(self):
        url = input("연습할 춤의 유튜브 링크: ")
        yt = pytube.YouTube(url)
        stream = yt.streams.filter(res="720p").first()
        stream.download(self.__download_path)
    
    def scaling_coor(self, keypoint_path): # "./keypoint_extraction/[주간아 직캠] IVE YUJIN - LOVE DIVE (아이브 유진 - 러브 다이브) l EP556_keypoints.json"
        with open(keypoint_path, "r") as file:
            data = json.load(file)
        pose_cor = pd.DataFrame(data["pose"])
        return np.array(pose_cor)
    
    def extract_keypoint(self, video_path):
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_style = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic
        
        keypoint_dict_pose = []
        keypoint_dict_left_hand = []
        keypoint_dict_right_hand = []
        keypoint_dict = {}
        
        try: cap = cv2.VideoCapture(video_path)
        except: return -1
        
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                success, image = cap.read()
                if not success: break
                self.__dance_shape = image.shape
                self.__dance_fps = cap.get(cv2.CAP_PROP_FPS)
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                try:
                    keypoint_dict_pose.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.pose_landmarks.landmark)})
                    keypoint_dict_left_hand.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.left_hand_landmarks.landmark)})
                    keypoint_dict_right_hand.append({str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.right_hand_landmarks.landmark)})
                except:
                    keypoint_dict_pose.append({str(idx): [0, 0, 0] for idx in range(0, 33)})
                    keypoint_dict_left_hand.append({str(idx): [0, 0, 0] for idx in range(0, 21)})
                    keypoint_dict_right_hand.append({str(idx): [0, 0, 0] for idx in range(0, 21)})
                    
                keypoint_dict = {"pose": keypoint_dict_pose, "left_hand": keypoint_dict_left_hand, "right_hand": keypoint_dict_right_hand}
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        with open(self.__keyporint_extraction_path+video_path.split("/")[2].split(".")[0]+"_keypoints.json", "w") as fp:
            json.dump(keypoint_dict, fp)
            
    def show_dance_tutorial(self):
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_style = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic
        
        try: cap = cv2.VideoCapture(0)
        except: cap = cv2.VideoCapture(1)
        load_dance = cv2.VideoCapture("./video/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561.mp4")
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        dance_cors = self.scaling_coor("./keypoint_extraction/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561_keypoints.json")
        idx = 0
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                success, image = cap.read()
                ret, dance = load_dance.read()
                if not success: break
                self.__user_shape = image.shape
                self.__user_fps = cap.get(cv2.CAP_PROP_FPS)
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                try:
                    for pose_point in range(33):
                        # scale_x_cor_pose, scale_y_cor_pose = int(dance_cors[idx][pose_point][0]*self.__dance_shape[1]), int(dance_cors[idx][pose_point][1]*self.__dance_shape[0])
                        scale_x_cor_pose, scale_y_cor_pose = int(dance_cors[idx][pose_point][0]*480), int(dance_cors[idx][pose_point][1]*720)
                        cv2.circle(image, (scale_x_cor_pose, scale_y_cor_pose), 5, (255, 102, 102), cv2.FILLED)
                    idx += 1
                except:
                    break
                # TODO: 싱크 문제 해결
                # try:
                #     h_output = np.hstack((cv2.flip(dance, 1), image))
                # except:
                #     h_output = np.hstack((np.zeros((512, 512, 3), np.uint8), image))
                cv2.imshow("Just DDance!", image)
                if cv2.waitKey(1)&0xFF==ord("q"): break
        load_dance.release()        
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
                


In [23]:
jd = JustDDance()

# jd.extract_keypoint("./video/[주간아 직캠] IVE YUJIN - LOVE DIVE (아이브 유진 - 러브 다이브) l EP556.mp4")

In [10]:
jd.extract_keypoint("./video/[주간아 직캠] LE SSERAFIM KIM CHAEWON - FEARLESS (르세라핌 김채원 - 피어리스) l EP561.mp4")

In [24]:
jd.show_dance_tutorial()